<a href="https://colab.research.google.com/github/KaikeDourado/DecisionTree_PredicaoDeEvasao/blob/main/Decision_Tree_Manual_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação Manual de Árvore de Decisão (CART) para Previsão de Evasão Escolar

Este notebook apresenta a implementação manual do algoritmo de Árvore de Decisão (Classification and Regression Trees - CART).

## 1. Configuração e Carregamento de Dados

Importamos a biblioteca `pandas` para manipulação de dados e `numpy` para operações numéricas.

In [1]:
import pandas as pd
import numpy as np
import json

# Carregar o dataset limpo
file_path = 'data.csv'
df = pd.read_csv(file_path, sep=';')

# Corrigir nome da coluna com espaço em branco e remover caractere especial
if df.columns[0].startswith('\ufeff'):
    df.rename(columns={df.columns[0]: df.columns[0].lstrip('\ufeff')}, inplace=True)
df.rename(columns={'Daytime/evening attendance\t': 'Daytime/evening attendance'}, inplace=True)

print("Primeiras 5 linhas do dataset:")
print(df.head())

Primeiras 5 linhas do dataset:
   Marital status  Application mode  Application order  Course  \
0               1                17                  5     171   
1               1                15                  1    9254   
2               1                 1                  5    9070   
3               1                17                  2    9773   
4               2                39                  1    8014   

   Daytime/evening attendance  Previous qualification  \
0                           1                       1   
1                           1                       1   
2                           1                       1   
3                           1                       1   
4                           0                       1   

   Previous qualification (grade)  Nacionality  Mother's qualification  \
0                           122.0            1                      19   
1                           160.0            1                       1   
2      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Implementação Manual do Algoritmo CART

O algoritmo CART (Classification and Regression Trees) é um método de aprendizado supervisionado que constrói árvores de decisão binárias.

### 2.1. Índice de Gini

O Índice de Gini mede a **impureza** de um nó. Um nó é puro (Gini = 0) se todas as amostras pertencerem à mesma classe.

In [2]:
def gini_index(groups, classes):
    """
    Calcula o Índice de Gini para um conjunto de grupos de dados.
    """
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        # Calcula a proporção de cada classe (p^2)
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # Pondera o Gini do grupo pelo seu tamanho (size / n_instances)
        gini += (1.0 - score) * (size / n_instances)
    return gini

### 2.2. Encontrando a Melhor Divisão (Split)

A função `get_best_split` itera sobre todas as colunas (atributos) e todos os valores únicos dentro de cada coluna para encontrar o ponto de corte que resulta no menor Índice de Gini.

In [3]:
def test_split(index, value, dataset):
    """
    Divide um dataset em dois grupos (esquerda e direita) com base em um atributo (índice) e um valor de divisão.
    """
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_best_split(dataset):
    """
    Encontra o melhor ponto de divisão para um dataset.
    """
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None

    # Itera sobre cada atributo (coluna) - a última coluna é a Target
    for index in range(len(dataset[0])-1):
        # Itera sobre cada valor único do atributo como um potencial ponto de corte
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)

            # Se o Gini for menor, encontramos um split melhor
            if gini < b_score:
                b_score = gini
                b_index = index
                b_value = row[index]
                b_groups = groups

    return {'index': b_index, 'value': b_value, 'groups': b_groups}


### 2.3. Construção da Árvore (Funções Recursivas)

A construção da árvore é feita de forma recursiva, parando quando as condições de parada (profundidade máxima ou tamanho mínimo do nó) são atingidas. Um nó terminal (`to_terminal`) é criado com a classe majoritária do grupo.

In [4]:
def to_terminal(group):
    """
    Cria um nó folha (terminal) com a classe mais comum no grupo.
    """
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    """
    Função recursiva para construir a árvore de decisão.
    """
    left, right = node['groups']
    del node['groups'] # Remove os dados para economizar memória

    # 1. Verifica se atingiu a profundidade máxima ou se o nó é puro
    if not left or not right or depth >= max_depth:
        node['left'] = to_terminal(left + right)
        node['right'] = to_terminal(left + right)
        return

    # 2. Processa o filho da esquerda
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_best_split(left)
        split(node['left'], max_depth, min_size, depth + 1)

    # 3. Processa o filho da direita
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_best_split(right)
        split(node['right'], max_depth, min_size, depth + 1)

def build_tree(train, max_depth, min_size):
    """
    Inicia a construção da árvore.
    """
    root = get_best_split(train)
    split(root, max_depth, min_size, 1)
    return root

## 3. Treinamento e Visualização da Árvore

Para o treinamento, usamos apenas as colunas numéricas do dataset, pois a implementação manual de divisões para dados categóricos é mais complexa. A variável alvo (`Target`) é mantida como string para a classificação.

**Parâmetros de Treinamento:**
*   **Profundidade Máxima (`MAX_DEPTH`):** 5 (para uma árvore interpretável)
*   **Tamanho Mínimo do Nó (`MIN_SIZE`):** 10 (para evitar overfitting)

In [5]:
# Preparação final dos dados para o algoritmo
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
dataset = df[numeric_cols + ['Target']].values.tolist()

# Mapeamento de índices para nomes de colunas (para visualização)
col_names = numeric_cols

# Parâmetros
MAX_DEPTH = 5
MIN_SIZE = 10

# Construir a árvore
tree = build_tree(dataset, MAX_DEPTH, MIN_SIZE)

print(f"Árvore construída com Profundidade Máxima={MAX_DEPTH} e Tamanho Mínimo do Nó={MIN_SIZE}.")

Árvore construída com Profundidade Máxima=5 e Tamanho Mínimo do Nó=10.


### 3.1. Visualização da Estrutura da Árvore

Abaixo está a representação textual da árvore de decisão gerada. Cada linha representa um nó, e a indentação indica a profundidade. A condição de divisão é mostrada como `[Nome_da_Coluna < Valor_de_Corte]`.

In [6]:
def print_tree(node, depth=0):
    """
    Imprime a estrutura da árvore de forma recursiva.
    """
    if isinstance(node, str):
        return f"{'  ' * depth}Previsão: {node}"

    # Mapear o índice da coluna para o nome da coluna
    col_name = col_names[node['index']]
    output = f"{'  ' * depth}[{col_name} < {node['value']:.2f}]"

    output += "\n" + print_tree(node['left'], depth + 1)
    output += "\n" + print_tree(node['right'], depth + 1)
    return output

print(print_tree(tree))

[Curricular units 2nd sem (approved) < 4.00]
  [Curricular units 2nd sem (approved) < 2.00]
    [Curricular units 1st sem (enrolled) < 1.00]
      [Admission grade < 136.10]
        [Tuition fees up to date < 1.00]
          Previsão: Dropout
          Previsão: Dropout
        [Tuition fees up to date < 1.00]
          Previsão: Graduate
          Previsão: Graduate
      [Mother's occupation < 134.00]
        [Curricular units 2nd sem (approved) < 1.00]
          Previsão: Dropout
          Previsão: Dropout
        Previsão: Enrolled
    [Tuition fees up to date < 1.00]
      [Curricular units 2nd sem (evaluations) < 18.00]
        [Admission grade < 138.80]
          Previsão: Dropout
          Previsão: Dropout
        Previsão: Enrolled
      [Age at enrollment < 23.00]
        [Curricular units 1st sem (approved) < 6.00]
          Previsão: Enrolled
          Previsão: Enrolled
        [Mother's occupation < 151.00]
          Previsão: Dropout
          Previsão: Dropout
  [Curr

**Análise da Árvore Gerada:**

O nó raiz da árvore é baseado na variável **`Curricular units 2nd sem (approved)`** (Unidades Curriculares Aprovadas no 2º Semestre). Isso sugere que o desempenho acadêmico no segundo semestre é o fator isolado mais importante para prever a evasão, o que está em linha com a literatura de Machine Learning em educação.

A primeira divisão é: `[Curricular units 2nd sem (approved) < 4.00]`.
*   Se o aluno aprovou **menos de 4** UCs no 2º semestre, ele segue para o ramo da esquerda (maior probabilidade de Evasão/Matriculado).
*   Se o aluno aprovou **4 ou mais** UCs, ele segue para o ramo da direita (maior probabilidade de Graduação).

Outras variáveis importantes que aparecem nos nós subsequentes incluem:
*   **`Tuition fees up to date`** (Propinas em dia): Um fator financeiro/administrativo crucial.
*   **`Admission grade`** (Nota de Admissão): Um indicador inicial de aptidão.
*   **`Curricular units 1st sem (enrolled)`** (UCs Matriculadas no 1º Semestre).

### 3.2. Previsão e Avaliação

As funções de previsão e avaliação demonstram como a árvore é usada para classificar novas amostras e medir sua precisão.

In [7]:
def predict(node, row):
    """
    Faz uma previsão para uma nova amostra, percorrendo a árvore.
    """
    # Se for um nó folha, retorna a previsão
    if isinstance(node, str):
        return node

    # Verifica a condição de divisão
    if row[node['index']] < node['value']:
        return predict(node['left'], row)
    else:
        return predict(node['right'], row)

def evaluate_tree(tree, dataset):
    """
    Avalia a precisão da árvore em um dataset.
    """
    correct = 0
    for row in dataset:
        prediction = predict(tree, row)
        if prediction == row[-1]:
            correct += 1
    return correct / len(dataset) * 100.0

# Avaliar a árvore (usando o dataset de treinamento como teste para demonstração)
accuracy = evaluate_tree(tree, dataset)

print(f"Precisão da Árvore de Decisão Manual no conjunto de treinamento: {accuracy:.2f}%")

# Exemplo de Previsão para a primeira linha do dataset
first_row = dataset[0]
prediction = predict(tree, first_row)
actual = first_row[-1]

print(f"\nExemplo de Previsão (Primeira Amostra):")
print(f"Valor Real: {actual}")
print(f"Previsão do Modelo: {prediction}")

Precisão da Árvore de Decisão Manual no conjunto de treinamento: 75.45%

Exemplo de Previsão (Primeira Amostra):
Valor Real: Graduate
Previsão do Modelo: Graduate
